In [ ]:
#Hi, Oliver. Pls just input the html to the second function 'extract_inference.' The first function is just a preparation part of the second function

In [571]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import random
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

In [572]:
f=pd.read_csv('word_frequency.csv')

In [575]:
def update_frequcny(html_file):
    kwords=[]
    file = open(html_file, "r")
    ad=file.read()
    regex='<div class="c7O9k">((\w| |-|:)+)'
    a=re.findall(regex,ad)
    #turn into list
    lst=[]
    lst2=[]
    num=0
    for ele in a:
        lst.append([ele[0]][0].strip())
    #remove basic information
    substring = ':'
    for ele in lst:
        if ele.startswith('Male')==False and ele.startswith('Female')==False and ele.endswith('years old')==False and (substring in ele)==False:
            lst2.append(ele)
    lst_first_appear=[]
    #find out word appearing for the first time
    for ele in lst2:
        if ele not in list(f['word']):
            lst_first_appear.append(ele)
    lst3=lst_first_appear
    #Google trend key word list prep
    if len(lst3)>=5:
        for num in range(5):
            word=random.choice(lst3)
            lst3.remove(word)
    else:
        kwords=lst_first_appear
    kwords.append('Used Vehicles')
    #Google trend api
    dict_google=dict(pytrends.get_historical_interest(kwords, year_start=2022, month_start=3, day_start=6, hour_start=0, year_end=2022, month_end=3, day_end=8).iloc[-1])
    #normalize the value
    del dict_google['isPartial']
    if dict_google['Used Vehicles']==0:
        dict_google['Used Vehicles']=1
    for key in dict_google.keys():
        num=int(dict_google[key])*67/dict_google['Used Vehicles']
        dict_google[key]=num
    #Value for existed tags
    existed_dict={}
    for ele in lst2:
        if ele in list(f['word']):
            existed_dict[ele]=list(f.loc[f['word'] == ele]['Google_Trend _Frequency'])[0]
    if len(dict_google)!=0:
        for key in dict_google.keys():
            existed_dict[key]=dict_google[key]
    return existed_dict

In [576]:
update_frequcny('nia_ads.html')

The request failed: Google returned a response with code 400.


{'Apparel': 670.0,
 'Autos': 11136.88889,
 'Beauty': 3082.0,
 'Books': 10227.7551,
 'Business News': 118.5384615,
 'Business Services': 237.5454545,
 'Camera': 4020.0,
 'Career Resources': 8.933333333,
 'Celebrities': 964.8,
 'Classical Music': 168.5806452,
 'Combat Sports': 2.68,
 'Computers': 3953.0,
 'Dogs': 19932.5,
 'Fashion': 2345.0,
 'Flowers': 3900.684932,
 'Food': 24407.14286,
 'Greeting Cards': 57.42857143,
 'Home Improvement': 614.1666667,
 'Local News': 475.0909091,
 'Mass Merchants': 0.2,
 'Movies': 27805.0,
 'Moving': 2881.0,
 'Music': 10440.83333,
 'Performing Arts': 137.5,
 'Pets': 5762.0,
 'Photographic': 30.0,
 'Politics': 552.75,
 'Pop Music': 79.88461538,
 'Restaurants': 31512.33333,
 'Romance Films': 1.810810811,
 'Shopping': 2345.0,
 'Sports': 5360.0,
 'Travel': 5695.0,
 'TV Dramas': 35.0,
 'TV Family-Oriented Shows': 0.022727273,
 'TV Game Shows': 14.48648649,
 'TV Reality Shows': 21.72972973,
 'TV Talk Shows': 3.621621622,
 'Urban Transit': 1.0,
 'Visual Art': 2

In [545]:
def extract_inference(html_file):
    #extract_tags
    file = open(html_file, "r")
    ad=file.read()
    regex='<div class="c7O9k">((\w| |-|:)+)'
    a=re.findall(regex,ad)
    #turn into list
    lst=[]
    num=0
    for ele in a:
        lst.append([ele[0]][0].strip())
    #frequency_dict
    frq_dic=update_frequcny(html_file)
    #extract specific tags we want
    age=''
    gender=''
    music=[]
    income=''
    marrial_status=''
    parental_status=''
    for i in lst:
        if i.startswith('Household Income: '):
            income=i.split(': ')[1].lower()
        if i.startswith('Marital Status: '):
            marrial_status=i.split(': ')[1].lower()
        if i.startswith('Parental Status: '):
            parental_status=i.split(': ')[1].lower()
        if i.endswith('Music'):
             music.append(i)
        if i.endswith('years old'):
            age=i.lower()
        if i=='Male'or i=='Female':
            gender=i.lower()
    for ele in music:
        if ele=='Music':
            music.remove('Music')
    games=[i for i in lst if i.endswith('Games')]
    for ele in music:
        if ele=='Games':
            games.remove('Games')
    lst_2=[]
    for ele in lst[2:]:
        if ':' not in ele:
            if ele not in games and ele not in music and ele!='Music' and ele!='Game':
                lst_2.append(ele.lower())
    #lst_habit=[]
    #num_list=[]
    #for num in range(7):    
    #    ind=random.randrange(len(lst_2))
    #    if ind in num_list:
    #        num=num-1
    #    else:
    #        num_list.append(ind)
    #        lst_habit.append(lst_2[ind].lower())
    lst_habit=[]
    if len(frq_dic)>=5:
        for num in range(5):
            a=min(frq_dic, key=frq_dic.get)
            b=a.lower()
            lst_habit.append(b)
            del frq_dic[a]
    else:
        for num in range(len(frq_dic)):
            a=min(frq_dic, key=frq_dic.get)
            b=a.lower()
            lst_habit.append(b)
            del frq_dic[a]
    #write basic information
    if income!=None:
        if age!='' and gender!='':
            final_str="According to the data Google extract from your activties, Google infers that you are a " +age+' '+gender+' with a '+income+' family income. '
        elif age=='' and gender!='':
            final_str="According to the data Google extract from your activties, Google infers that you are a "+gender+' with a '+income+' family income. '
        elif age!='' and gender=='':
            final_str="According to the data Google extract from your activties, Google infers that you are " +age+' with a '+income+' family income. '
        elif age=='' and gender=='':
            final_str="According to the data Google extract from your activties, Google infers that you have a " +income+' family income. '
    elif income==None:
        if age!='' and gender!='':
            final_str="According to the data Google extract from your activties, Google infers that you are a " +age+' '+gender+'. '
        elif age!='' and gender=='':
            final_str="According to the data Google extract from your activties, Google infers that you are " +age+'. '
        elif age=='' and gender!='':
            final_str="According to the data Google extract from your activties, Google infers that you are a "+gender+'. '
        else:
            final_str=''
    if parental_status!='' and marrial_status!='':
        final_str=final_str+ 'Google also infers that you are '+marrial_status+' and '+parental_status+'. '
    if parental_status=='' and marrial_status!='':
        final_str=final_str+ 'Google also infers that you are '+marrial_status+'. '
    if parental_status!='' and marrial_status=='':
        final_str=final_str+ 'Google also infers that you are '+parental_status+'. '
    #write habits
    if len(lst_habit)>4:
        final_str2='From your activties and searches on Google, Google infers that you may be interested in '+lst_habit[0]+' and '+ lst_habit[1]+'. '
        final_str2+='Google also noticed that you are interested in '+lst_habit[2]+', '+ lst_habit[3]+' and '+lst_habit[4]+'. '
    elif len(lst_habit)==4:
        final_str2='From your activties and searches on Google, Google infers that you may be interested in '+lst_habit[0]+' and '+ lst_habit[1]+'. '
        final_str2+='Google also noticed that you are interested in '+lst_habit[2]+' and '+lst_habit[3]+'. '
    elif len(lst_habit)==3:
        final_str2='From your activties and searches on Google, Google infers that you may be interested in '+lst_habit[0]+', '+ lst_habit[1]+' and '+ lst_habit[2]+'. '
    elif len(lst_habit)==2:
        final_str2='From your activties and searches on Google, Google infers that you may be interested in '+lst_habit[0]+' and '+ lst_habit[1]+'. '
    elif len(lst_habit)==1:
        final_str2='From your activties and searches on Google, Google notices that you may be interested in '+lst_habit[0]+'. '
    elif len(lst_habit)==0:
        final_str2='Google knows nothing about your habits because of your limited activty on Google. '
    #write catgorical habits
    if len(music)>1:
        final_str2+='Among different types of musics, you enjoyed listening to'
        for ele in music:
                if ele!=music[-1]:
                    final_str2=final_str2+' '+ele.lower()+','
                else:
                    final_str2=final_str2+' and '+ele.lower()+'. '
    if len(games)>1:
        final_str2+='You also enjoy playing'
        for ele in games:
            if ele!=games[-1]:
                final_str2=final_str2+' '+ele.lower()+','
            else:
                final_str2=final_str2+' and '+ele.lower()+'.'
    return final_str+final_str2

In [580]:
extract_inference("Ad Settings.html")

The request failed: Google returned a response with code 400.


'According to the data Google extract from your activties, Google infers that you are a 18-34 years old male with a upper middle family income. Google also infers that you are single and not a parent. From your activties and searches on Google, Google infers that you may be interested in mass merchants and roleplaying games. Google also noticed that you are interested in urban transit, tv sci-fi and combat sports. Among different types of musics, you enjoyed listening to classical music, pop music, and world music. You also enjoy playing adventure games, roleplaying games, and strategy games.'

In [581]:
extract_inference("amirul_ads.html")

The request failed: Google returned a response with code 400.


'According to the data Google extract from your activties, Google infers that you are a 18-24 years old male with a lower middle family income. Google also infers that you are single and not a parent. From your activties and searches on Google, Google infers that you may be interested in tv family-oriented shows and shopping portals. Google also noticed that you are interested in mass merchants, event ticket sales and multimedia software. Among different types of musics, you enjoyed listening to classical music, pop music, and world music. You also enjoy playing adventure games, massively multiplayer games, roleplaying games, and strategy games.'

In [582]:
extract_inference("oliver_ads.html")

The request failed: Google returned a response with code 400.


'According to the data Google extract from your activties, Google infers that you have a upper middle family income. Google also infers that you are not a parent. From your activties and searches on Google, Google infers that you may be interested in urban transit and business formation. Google also noticed that you are interested in career resources, computer hardware and used vehicles. '

In [583]:
extract_inference("nia_ads.html")

The request failed: Google returned a response with code 400.


'According to the data Google extract from your activties, Google infers that you are a 18-44 years old female with a high family income. Google also infers that you are married and not a parent. From your activties and searches on Google, Google infers that you may be interested in tv family-oriented shows and mass merchants. Google also noticed that you are interested in urban transit, romance films and combat sports. Among different types of musics, you enjoyed listening to classical music, and pop music. '

In [584]:
extract_inference("tanbirul_ads.html")

The request failed: Google returned a response with code 400.


'According to the data Google extract from your activties, Google infers that you are a 35-44 years old male with a upper middle family income. Google also infers that you are single and not a parent. From your activties and searches on Google, Google infers that you may be interested in mass merchants and event ticket sales. Google also noticed that you are interested in price comparisons, vehicle shopping and roleplaying games. Among different types of musics, you enjoyed listening to pop music, rock music, and world music. You also enjoy playing adventure games, casual games, family-oriented games, massively multiplayer games, roleplaying games, shooter games, sports games, and strategy games.'